In [1]:
# import the necessary libraries
from pyforest import *
import numpy_financial as npf
from dateutil.relativedelta import relativedelta
from datetime import date
import pandas as pd 
# import streamlit as st #for frontend
# app pour devis et facture mobile 
# avec envois automatique

In [2]:
# Une feuille par controle de parametre 
# sort the file by date not alpha col = EcritureDate X
# toutes les operations en cash superieures a 5K et total de ces operations
# mettre toutes ces operations sur une feuille excel
# CompteNum 531 = cash 
# CompteNum 471 = a mettre aussi 
# Loi statistique sur utilisation des chiffres recurrent 
# TVA recuperable - montant superieur comparant la cell total et la cell TVA recup 
# CompteNum TVA Mc = 4456610000 --- seulement les 3 premiers chiffres 
# CompteNum TVA FR = 4456620000
# Compte 531 - solde progressif qui soit pas rouge et jamais crediteur - Nouvelle fiche
# 10 plus grosses operations
# for each compte num convert to str and use str contains

In [3]:
# get data from excel function
def get_data_from_excel(sheet, excelFileName):
    path_excel = excelFileName 
    df = pd.read_excel(
        io = path_excel,
        engine = 'openpyxl',
        sheet_name = sheet)
    return df

# function for process
def process(excel_name, fec='FEC', Param='Param'): 
    # read in excel
    param = get_data_from_excel(Param, excel_name)

    # assign variables to the parameters
    date = param['Date'][0]
    company = param['Entreprise'][0]
    montantcash = param['Cash'][0]

    # read in FEC or read in as csv 
    df = get_data_from_excel(fec, excel_name)
        
    # sort df by date
    df.sort_values(by = 'EcritureDate', inplace = True, ascending = True)
    # get 10 largest txs
    df['Montant'] = df['Montant'].str.replace(',', '.').astype(float)
    big10 = df.nlargest(n=10, columns=['Montant'])
    # get cash acct convert to str  Commence par 531 pas contains
    df['CompteNum'] = df['CompteNum'].astype(str)
    cash = df[df['CompteNum'].str.contains('531')]
    # calculate progressive cash account
    # if cash n+1 > n-1 flag those 2 txs
    # add if for dates if they are all the same
    if (cash.EcritureDate.eq(cash.EcritureDate.iloc[0]).all()) == True: 
        cash['AcctEspeces'] = 0
    else:
        cash['AcctEspeces'] = cash['Montant'].shift(-1)>cash['Montant']
    # get cash tx larger than 5K
    hcash = cash.loc[(df['Montant'] > montantcash)]
    # get total cash txs
    cash['Total Toutes Especes'] = cash['Montant'].sum()
    # get TVA FR and MC and compare tva amount with purchase amount
    tvafr = df[df['CompteNum'].str.contains('445662')]
    tvafr = tvafr.loc[tvafr['Montant'] > tvafr['Montant'][-1]]

    tvamc = df[df['CompteNum'].str.contains('445661')]
    tvamc = tvamc.loc[tvamc['Montant'] > tvamc['Montant'][-1]]

    # move the different dfs to different excel sheets
    writer = pd.ExcelWriter(f'FEC_{company}_{date}.xlsx', engine='xlsxwriter')
    tvafr.to_excel(writer, sheet_name = 'TVA FR')
    tvamc.to_excel(writer, sheet_name = 'TVA MC')
    big10.to_excel(writer, sheet_name = '10+ Operations')
    hcash.to_excel(writer, sheet_name = f'Especes {montantcash}+')
    cash.to_excel(writer, sheet_name = 'Compte Cash')
    
    writer.save()

process('fec_param.xlsx')

In [5]:
fec='FEC'
Param='Param'
excel_name = 'fec_param.xlsx'

# read in excel
param = get_data_from_excel(Param, excel_name)

# assign variables to the parameters
date = param['Date'][0]
company = param['Entreprise'][0]
montantcash = param['Montant Cash'][0]

# read in FEC or read in as csv 
df = get_data_from_excel(fec, excel_name)
# sort df by date
df.sort_values(by = 'EcritureDate', inplace = True, ascending = True)

# get 10 largest txs
df['Montant'] = df['Montant'].astype(str)
df['Montant'] = df['Montant'].str.replace(',', '.')
df['Montant'] = df['Montant'].astype(float)
big10 = df.nlargest(n=10, columns=['Montant'])


In [6]:
df['CompteNum'] = df['CompteNum'].astype(str)
cash = df[df['CompteNum'].str.contains('531')]
    # calculate progressive cash account
    # if cash n+1 > n-1 flag those 2 txs
cash['AcctEspeces'] = np.where(df['column'] == 'condition', 'result if yes', 'result if no')

<ipython-input-6-5697b252fb30>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash['AcctEspeces'] = cash['Montant'].shift(-1)>cash['Montant']


In [ ]:
""" where cell ==  531 in CompteNum col
loc cell >4999 in montant col

where comptenum == 4456610000 or 4456620000
loc if montant TVA > cell total (one right above)

"""
